# Interactive Tutorial: DBSI Fitting

This notebook guides you through using the `dbsi_toolbox` to load DWI data, fit the DBSI model, and save the results.

## Setup and Imports

Ensure the toolbox is installed (run `pip install -e .` from the repository root). Let's import the necessary modules.

In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import sys
from time import time

# Import from our toolbox
try:
    from dbsi_toolbox.utils import load_dwi_data_dipy, save_parameter_maps
    from dbsi_toolbox import DBSIModel
except ImportError:
    print("ERROR: Could not import 'dbsi_toolbox'.", file=sys.stderr)
    print("Make sure you have installed the package by running 'pip install -e .' from the root directory.", file=sys.stderr)

# Configure matplotlib for plotting
%matplotlib inline
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [12, 12]

## Define File Paths

Edit the paths below to point to your data.

In [ ]:
# --- EDIT THESE PATHS ---
f_corrected = '<your_corrected_dwi_file_path.nii.gz>'
fbval = '<your_bval_file_path>'
fbvec = '<your_bvec_file_path.bvec>'
f_mask = '<your_brain_mask_file_path.nii.gz>'
output_dir = '<your_output_directory_folder_path>'
prefix = "<id_prefix_for_output_files>"

## Load Data

We'll use the `load_dwi_data_dipy` utility function to load everything.

In [ ]:
try:
    data, affine, gtab, mask = load_dwi_data_dipy(
        f_nifti=f_corrected,
        f_bval=fbval,
        f_bvec=fbvec,
        f_mask=f_mask
    )
    print(f"\n✓ Data loaded. Voxels in mask: {np.sum(mask):,}")
except FileNotFoundError as e:
    print(f"ERROR: File not found. Check your paths.\n{e}", file=sys.stderr)

## Initialize and Fit the Model

We'll create an instance of `DBSIModel` and run `fit_volume`. This is the most time-consuming step.

In [ ]:
import numpy as np
import os
from dbsi_toolbox.utils import save_parameter_maps
from dbsi_toolbox.twostep import DBSI_TwoStep
from dbsi_toolbox.calibration import optimize_dbsi_params

# Extract b-values and b-vectors from the loaded gradient table
real_bvals = gtab.bvals
real_bvecs = gtab.bvecs # Shape (N, 3)

print(f"Protocol detected: {len(real_bvals)} volumes.")

# ==========================================
# AUTOMATIC HYPERPARAMETER CALIBRATION
# ==========================================
# This step performs an In-Silico Monte Carlo simulation using 
# the EXACT geometry (b-vecs) and b-values of your real scan.
# It finds the best trade-off between spectral resolution and stability.

print("\n--- Starting Auto-Calibration ---")
best_params = optimize_dbsi_params(
    real_bvals, 
    real_bvecs,
    snr_estimate=30.0,       # Conservative estimate for 3T clinical scans
    n_monte_carlo=1000,      # High iterations for statistical robustness
    bases_grid=[25, 50, 75], # Test ranges for isotropic resolution
    lambdas_grid=[0.01, 0.1, 0.5] # Test ranges for regularization
)

optimal_bases = best_params['n_bases']
optimal_lambda = best_params['reg_lambda']

print(f"\nCalibration Complete. Optimized Parameters:")
print(f" -> Isotropic Bases: {optimal_bases}")
print(f" -> Regularization (Lambda): {optimal_lambda}")

# ==========================================
# INITIALIZE & FIT MODEL
# ==========================================
print("\n--- Initializing Two-Step DBSI Model ---")

model = DBSI_TwoStep(
    n_iso_bases=optimal_bases,
    reg_lambda=optimal_lambda,
    iso_diffusivity_range=(0.0, 3.0e-3)
)

print("Running Volumetric Fit (This may take a while)...")
# Run the fit on the masked volume
maps = model.fit_volume(data, real_bvals, real_bvecs, mask=mask)


## Save the Maps

Finally, let's save all the output maps to NIfTI files.

In [ ]:
# ==========================================
# SAVE RESULTS
# ==========================================
print(f"\nSaving parameter maps to: {output_dir}")

save_parameter_maps(maps, affine, output_dir=output_dir, prefix=prefix)

print("Processing successfully finished.")